In [ ]:
from glob import glob
import xarray as xr
import cftime
import nc_time_axis
import numpy as np
import matplotlib.pyplot as plt
import intake, intake_esm
!pip install cmip6_preprocessing
from cmip6_preprocessing.preprocessing import (correct_units,rename_cmip6)

In [43]:
def chunk_time(ds):
    if 'time' in ds.dims:
        ds = ds.chunk({'time':1})
    return ds

In [44]:
# Necessary for creating a common time axis for all models
# We want to create a common time axis so there will be no gaps when plotting the results 

def fix_time(ds):
    """ force calendar to noleap"""
    import xarray as xr
    
    if "time" not in ds.dims:
        return ds
    
    if ("calendar" not in ds["time"].attrs): 
        ds["time"].attrs.update({"calendar": "noleap"})
        
    if ds["time"].attrs["calendar"] not in ["noleap", "NOLEAP", "365_day"]:
        ds["time"].attrs.update({"calendar": "noleap"})
        
    ds = xr.decode_cf(ds)
    return ds

In [45]:
# Pass this function for preprocessing thetao data 
def pp_thetao(ds):
    ds = rename_cmip6(ds)
    ds = fix_time(ds)
    ds = correct_units(ds)
    return ds

In [46]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 


In [47]:
col = intake.open_esm_datastore(col_url)

In [48]:
esmcol_data = col.esmcol_data
testds = "CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon"

In [49]:
cat_T = col.search(experiment_id=['historical'],
                 mip_table='Omon',
                 ensemble_member=["r1i1p1f1"],
                 model=['IPSL-CM6A-LR'],
                 #model=['GFDL-CM4','GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["thetao"])
               #  version=['v20190308', 'v20191120', 'v20200220','v20191108','v20190323','v20190731','v20190710','v20180803',
                #          'v20191007','v20190627','v20190311','v20190429','v20190627','v20191205','v20190914','v20190815','v20191108'])


In [51]:
cat_T.df.groupby(['model']).nunique()#CanESM5 has two versions

,project,institute,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
model,,,,,,,,,,,,
IPSL-CM6A-LR,1,1,1,1,1,1,1,1,1,2,1,2


In [53]:
def latest_version(ds):
    """filters latest DRS versions of datasets only"""
    ds=ds.df.sort_values(['version']).groupby(['temporal subset','model','mip_table','institute','variable','ensemble_member','grid_label','experiment_id'],as_index=False).last()
    return ds

In [54]:
cat_T_new = latest_version(cat_T)

In [55]:
cat_T_new[cat_T_new['model']=='CanESM5'] #one distinct version only, latest one.

,temporal subset,model,mip_table,institute,variable,ensemble_member,grid_label,experiment_id,project,frequency,modeling_realm,version,path


In [56]:
cat_T_new

,temporal subset,model,mip_table,institute,variable,ensemble_member,grid_label,experiment_id,project,frequency,modeling_realm,version,path
0,185001-194912,IPSL-CM6A-LR,Omon,IPSL,thetao,r1i1p1f1,gn,historical,CMIP6,mon,ocean,v20180803,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/h...
1,195001-201412,IPSL-CM6A-LR,Omon,IPSL,thetao,r1i1p1f1,gn,historical,CMIP6,mon,ocean,v20180803,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/h...


In [57]:
cat_T = intake.open_esm_datastore(cat_T_new,esmcol_data=esmcol_data)

In [ ]:
dset_dict_T = cat_T.to_dataset_dict(cdf_kwargs={'decode_times': False, 'chunks': {'time': 1,'olevel':1}},
                                preprocess = pp_thetao,storage_options=dict(anon=True))

In [61]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon']

<xarray.Dataset>
Dimensions:          (bnds: 2, ensemble_member: 1, lev: 75, time: 1980, vertex: 4, x: 362, y: 332)
Coordinates:
    lat              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    lon              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * lev              (lev) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time             (time) object 1850-01-16 12:00:00 ... 2015-01-25 12:00:00
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds, vertex, x, y
Data variables:
    lon_bounds       (y, x, vertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
    lat_bounds       (y, x, vertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
    area             (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    lev_bounds       (lev, bnds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_bounds      (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    thetao           (ensemble_member, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 1, 332, 362), meta=np.ndarray>
Attributes:
    history:                 Sat Dec  1 12:16:38 2018: ncatted -O -a realizat...
    institution_id:          IPSL
    dr2xml_version:          1.11
    nominal_resolution:      100 km
    grid:                    native ocean tri-polar grid with 105 k ocean cells
    experiment:              all-forcing simulation of the recent past
    data_specs_version:      01.00.21
    branch_time_in_child:    [0.]
    source_type:             AOGCM BGC
    institution:             Institut Pierre Simon Laplace, Paris 75252, France
    parent_variant_label:    r1i1p1f1
    dr2xml_md5sum:           f1e40c1fc5d8281f865f72fbf4e38f9d
    title:                   IPSL-CM6A-LR model output prepared for CMIP6 / C...
    license:                 CMIP6 model data produced by IPSL is licensed un...
    external_variables:      areacello volcello
    NCO:                     "4.6.0"
    Conventions:             CF-1.7 CMIP-6.2
    branch_time_in_parent:   [21914.]
    realm:                   ocean
    initialization_index:    [1]
    parent_time_units:       days since 1850-01-01 00:00:00
    grid_label:              gn
    parent_mip_era:          CMIP6
    mip_era:                 CMIP6
    model_version:           6.1.5
    frequency:               mon
    sub_experiment_id:       none
    activity_id:             CMIP
    parent_experiment_id:    piControl
    physics_index:           [1]
    branch_method:           standard
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    description:             CMIP6 historical
    EXPID:                   historical
    table_id:                Omon
    experiment_id:           historical
    variable_id:             thetao
    parent_source_id:        IPSL-CM6A-LR
    tracking_id:             hdl:21.14100/2357970e-3f77-4595-80d8-e3d5c69d0bd...
    variant_label:           r1i1p1f1
    realization_index:       [1]
    source:                  IPSL-CM6A-LR (2017):  atmos: LMDZ (NPv6, N96; 14...
    sub_experiment:          none
    source_id:               IPSL-CM6A-LR
    name:                    /ccc/work/cont003/gencmip6/p86caub/IGCM_OUT/IPSL...
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.IPSL.IPSL-C...
    contact:                 ipsl-cmip6@listes.ipsl.fr
    intake_esm_varname:      ['thetao']
    forcing_index:           [1]
    product:                 model-output
    parent_activity_id:      CMIP
    intake_esm_dataset_key:  CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon

In [62]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon'].thetao

<xarray.DataArray 'thetao' (ensemble_member: 1, time: 1980, lev: 75, y: 332, x: 362)>
dask.array<broadcast_to, shape=(1, 1980, 75, 332, 362), dtype=float32, chunksize=(1, 1, 1, 332, 362), chunktype=numpy.ndarray>
Coordinates:
    lat              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    lon              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * lev              (lev) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time             (time) object 1850-01-16 12:00:00 ... 2015-01-25 12:00:00
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           Sea Water Potential Temperature
    units:               degC
    online_operation:    average
    cell_methods:        area: mean where sea time: mean
    interval_operation:  2700 s
    interval_write:      1 month
    cell_measures:       area: areacello volume: volcello
    description:         Diagnostic should be contributed even for models usi...
    history:             none

In [64]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon'].lev_bounds

<xarray.DataArray 'lev_bounds' (lev: 75, bnds: 2)>
dask.array<open_dataset-10a3a4952581771b3a6502de64a63405olevel_bounds, shape=(75, 2), dtype=float32, chunksize=(1, 2), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float32 0.50576 1.5558553 2.6676817 ... 5698.0605 5902.0576
Dimensions without coordinates: bnds
Attributes:
    units:    m

In [65]:
#another chunking test
dset_dict_T = cat_T.to_dataset_dict(cdf_kwargs={'decode_times': False, 'chunks': {'time': 1}},
                                preprocess = pp_thetao,storage_options=dict(anon=True))


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [66]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon']

<xarray.Dataset>
Dimensions:          (bnds: 2, ensemble_member: 1, lev: 75, time: 1980, vertex: 4, x: 362, y: 332)
Coordinates:
    lat              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    lon              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * lev              (lev) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time             (time) object 1850-01-16 12:00:00 ... 2015-01-25 12:00:00
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Dimensions without coordinates: bnds, vertex, x, y
Data variables:
    lon_bounds       (y, x, vertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
    lat_bounds       (y, x, vertex) float32 dask.array<chunksize=(332, 362, 4), meta=np.ndarray>
    area             (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    lev_bounds       (lev, bnds) float32 dask.array<chunksize=(75, 2), meta=np.ndarray>
    time_bounds      (time, bnds) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    thetao           (ensemble_member, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 75, 332, 362), meta=np.ndarray>
Attributes:
    history:                 Sat Dec  1 12:16:38 2018: ncatted -O -a realizat...
    institution_id:          IPSL
    dr2xml_version:          1.11
    nominal_resolution:      100 km
    grid:                    native ocean tri-polar grid with 105 k ocean cells
    experiment:              all-forcing simulation of the recent past
    data_specs_version:      01.00.21
    branch_time_in_child:    [0.]
    source_type:             AOGCM BGC
    institution:             Institut Pierre Simon Laplace, Paris 75252, France
    parent_variant_label:    r1i1p1f1
    dr2xml_md5sum:           f1e40c1fc5d8281f865f72fbf4e38f9d
    title:                   IPSL-CM6A-LR model output prepared for CMIP6 / C...
    license:                 CMIP6 model data produced by IPSL is licensed un...
    external_variables:      areacello volcello
    NCO:                     "4.6.0"
    Conventions:             CF-1.7 CMIP-6.2
    branch_time_in_parent:   [21914.]
    realm:                   ocean
    initialization_index:    [1]
    parent_time_units:       days since 1850-01-01 00:00:00
    grid_label:              gn
    parent_mip_era:          CMIP6
    mip_era:                 CMIP6
    model_version:           6.1.5
    frequency:               mon
    sub_experiment_id:       none
    activity_id:             CMIP
    parent_experiment_id:    piControl
    physics_index:           [1]
    branch_method:           standard
    CMIP6_CV_version:        cv=6.2.3.5-2-g63b123e
    description:             CMIP6 historical
    EXPID:                   historical
    table_id:                Omon
    experiment_id:           historical
    variable_id:             thetao
    parent_source_id:        IPSL-CM6A-LR
    tracking_id:             hdl:21.14100/2357970e-3f77-4595-80d8-e3d5c69d0bd...
    variant_label:           r1i1p1f1
    realization_index:       [1]
    source:                  IPSL-CM6A-LR (2017):  atmos: LMDZ (NPv6, N96; 14...
    sub_experiment:          none
    source_id:               IPSL-CM6A-LR
    name:                    /ccc/work/cont003/gencmip6/p86caub/IGCM_OUT/IPSL...
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.IPSL.IPSL-C...
    contact:                 ipsl-cmip6@listes.ipsl.fr
    intake_esm_varname:      ['thetao']
    forcing_index:           [1]
    product:                 model-output
    parent_activity_id:      CMIP
    intake_esm_dataset_key:  CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon

In [67]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon'].lev_bounds

<xarray.DataArray 'lev_bounds' (lev: 75, bnds: 2)>
dask.array<open_dataset-7b0243a3c755e2ddbd19e7d6fa8018f7olevel_bounds, shape=(75, 2), dtype=float32, chunksize=(75, 2), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float32 0.50576 1.5558553 2.6676817 ... 5698.0605 5902.0576
Dimensions without coordinates: bnds
Attributes:
    units:    m

In [68]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon'].lev

<xarray.DataArray 'lev' (lev: 75)>
array([5.057600e-01, 1.555855e+00, 2.667682e+00, 3.856280e+00, 5.140361e+00,
       6.543034e+00, 8.092519e+00, 9.822750e+00, 1.177368e+01, 1.399104e+01,
       1.652532e+01, 1.942980e+01, 2.275762e+01, 2.655830e+01, 3.087456e+01,
       3.574020e+01, 4.118002e+01, 4.721189e+01, 5.385064e+01, 6.111284e+01,
       6.902168e+01, 7.761116e+01, 8.692943e+01, 9.704131e+01, 1.080303e+02,
       1.200000e+02, 1.330758e+02, 1.474062e+02, 1.631645e+02, 1.805499e+02,
       1.997900e+02, 2.211412e+02, 2.448906e+02, 2.713564e+02, 3.008875e+02,
       3.338628e+02, 3.706885e+02, 4.117939e+02, 4.576256e+02, 5.086399e+02,
       5.652923e+02, 6.280260e+02, 6.972587e+02, 7.733683e+02, 8.566790e+02,
       9.474479e+02, 1.045854e+03, 1.151991e+03, 1.265861e+03, 1.387377e+03,
       1.516364e+03, 1.652568e+03, 1.795671e+03, 1.945296e+03, 2.101027e+03,
       2.262422e+03, 2.429025e+03, 2.600380e+03, 2.776039e+03, 2.955570e+03,
       3.138565e+03, 3.324641e+03, 3.513446e+03, 3.704657e+03, 3.897982e+03,
       4.093159e+03, 4.289953e+03, 4.488155e+03, 4.687581e+03, 4.888070e+03,
       5.089479e+03, 5.291683e+03, 5.494575e+03, 5.698061e+03, 5.902058e+03],
      dtype=float32)
Coordinates:
  * lev      (lev) float32 0.50576 1.5558553 2.6676817 ... 5698.0605 5902.0576
Attributes:
    name:       olevel
    long_name:  Vertical T levels
    units:      m
    positive:   down
    bounds:     olevel_bounds

In [69]:
dset_dict_T['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon'].thetao

<xarray.DataArray 'thetao' (ensemble_member: 1, time: 1980, lev: 75, y: 332, x: 362)>
dask.array<broadcast_to, shape=(1, 1980, 75, 332, 362), dtype=float32, chunksize=(1, 1, 75, 332, 362), chunktype=numpy.ndarray>
Coordinates:
    lat              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    lon              (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * lev              (lev) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
  * time             (time) object 1850-01-16 12:00:00 ... 2015-01-25 12:00:00
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           Sea Water Potential Temperature
    units:               degC
    online_operation:    average
    cell_methods:        area: mean where sea time: mean
    interval_operation:  2700 s
    interval_write:      1 month
    cell_measures:       area: areacello volume: volcello
    description:         Diagnostic should be contributed even for models usi...
    history:             none